In [1]:
import pandas as pd
df = pd.read_csv('../Data/lstm.csv')
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [2]:
df.category.unique()

array(['food', 'sports'], dtype=object)

> 중복된것을 제거하고 대소문자를 찢어놔야된다.


In [3]:
# 데이터에 사용된 중복 없는 전체 단어 갯수를 파악
results = set() # 중복 제거 : set()
df['paragraph'].str.lower().str.split().apply(results.update)
vocab_size = len(results)
vocab_size


536

In [4]:
# 단어를 숫자로 인코딩
paragraphs = df['paragraph'].to_list()
paragraphs[0:5]

['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [5]:
from tensorflow import keras
# keras에서 setseed한 것
# keras.utils.set_random_seed(1)

In [18]:
# One hot encoding
encoded_paragraphs = [keras.preprocessing.text.one_hot(paragraph, vocab_size) for paragraph in paragraphs]
print("one-hot-encoding[1] :",encoded_paragraphs[1],"\n전체 row 갯수 : ",len(encoded_paragraphs))

one-hot-encoding[1] : [533, 190, 523, 312, 213, 196, 527, 362, 242, 418, 190, 264, 266, 100, 377, 376, 94, 135, 441] 
전체 row 갯수 :  20


In [7]:
# 데이터에서 가장 긴 문장의 단어 갯수 확인
max_length = 0
for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
        max_length = len(row.split(" "))
print(max_length)    

91


In [8]:
# 문장마다 단어 갯수가 다르므로 sequencce padding을 넣어서 문장의 길이를 동일하게 만듭니다.
padded_paragraphs_encoding = keras.preprocessing.sequence.pad_sequences(
    encoded_paragraphs, maxlen = max_length, padding='post'
)
padded_paragraphs_encoding


array([[240, 149, 347, ...,   0,   0,   0],
       [533, 190, 523, ...,   0,   0,   0],
       [188, 338,  49, ...,   0,   0,   0],
       ...,
       [277, 518, 150, ...,   0,   0,   0],
       [362, 443, 113, ...,   0,   0,   0],
       [141, 446, 443, ...,   0,   0,   0]], dtype=int32)

In [9]:
# 분류항목(food, sports)를 수치로 변경하기

categories = df['category'].to_list()

def category_encode(category):
    if category == "food":
        return [1,0]
    else:
        return [0,1]

In [20]:
encoded_category = [category_encode(category) for category in categories]
encoded_category[:11]

[[1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1]]

In [11]:
# Feature 확인
print(encoded_paragraphs[0])

[240, 149, 347, 417, 83, 491, 107, 149, 232, 223, 443, 154, 362, 43, 190, 523, 429, 311, 166, 213, 190, 441, 527, 310, 280, 266, 76, 446, 311, 504, 79, 150, 261, 489, 529, 142, 248, 135, 251, 196, 488, 76, 412, 503, 369, 425, 243, 292, 460, 443, 413, 311, 344]


In [12]:
len(encoded_paragraphs[19])

73

---
### 주제를 분류하는 모델 구현하기

In [13]:
model = keras.Sequential()
# 문맥 생성 단계
model.add(keras.layers.Embedding(vocab_size, 5, input_length=max_length))
# embedding 레이어는 인덱스를 받아 5차원 벡터의 embedding을 출력합니다. 
# 5차원인 이유는 과거, 과거진행, 과거완료, 현재, 현재진행, 현재완료 ? 아님 5형식?
model.add(keras.layers.LSTM(64))
# 분류단계
model.add(keras.layers.Dense(32, activation='relu')) # LSTM 반값
model.add(keras.layers.Dense(2, activation='softmax')) # 문장은 확률값을 구해야하므로

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [15]:
import numpy as np
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [16]:
# 데이터가 많으면 batch_size를 써줘야된다.
model.fit(train_X, train_Y, batch_size=10, epochs=50)

Epoch 1/50


2022-07-29 09:30:39.124626: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 1s 10ms/step - loss: 0.6926 - accuracy: 0.5500
Epoch 2/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6926 - accuracy: 0.5500
Epoch 3/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6926 - accuracy: 0.5500
Epoch 4/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6909 - accuracy: 0.6000
Epoch 5/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6906 - accuracy: 0.6000
Epoch 6/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6902 - accuracy: 0.6000
Epoch 7/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6898 - accuracy: 0.6000
Epoch 8/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6886 - accuracy: 0.6000
Epoch 9/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6880 - accuracy: 0.6000
Epoch 10/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6881 - accuracy: 0.6000
Epoch 11/50
2/2 [=======================